## Download the dataset



In [0]:
!git clone https://github.com/mttgdd/RobotCarDataset-Scraper.git
!mkdir ./Downloads

In [13]:
!python RobotCarDataset-Scraper/scrape_mrgdatashare.py --downloads_dir Downloads --datasets_file RobotCarDataset-Scraper/demo.csv --username zirui --password 2251598zzrrywD

ScrapeMRGDatashare is starting...
reading datasets_file: /content/RobotCarDataset-Scraper/demo.csv
got num_datasets: 1
got csrf_middleware_token: 8ztVU9ZLCB1ZmVOmQcDU8k8i2Djlhv54j8ZplpGsCsq4p3cfEVlINKk6LVjluSd8
Logged in!
num_chunks_in_period: 0, period_seconds: 600
requesting file_url: http://mrgdatashare.robots.ox.ac.uk:80/download/?filename=datasets/2015-10-29-12-18-17/2015-10-29-12-18-17_tags.tar
downloading local_file_path: /content/Downloads/2015-10-29-12-18-17_tags.tar
100% 10.0/10.0 [00:00<00:00, 25.5kKB/s]
unzipping local_file_path: /content/Downloads/2015-10-29-12-18-17_tags.tar
requesting file_url: http://mrgdatashare.robots.ox.ac.uk:80/download/?filename=datasets/2015-10-29-12-18-17/2015-10-29-12-18-17_stereo_left_01.tar
downloading local_file_path: /content/Downloads/2015-10-29-12-18-17_stereo_left_01.tar
100% 4.46M/4.46M [02:37<00:00, 28.2kKB/s]
unzipping local_file_path: /content/Downloads/2015-10-29-12-18-17_stereo_left_01.tar
requesting file_url: http://mrgdatashare.ro

## Extract descriptor



In [0]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
import cv2
from PIL import Image


model_file = 'alexnet_places365.pth.tar'
if not os.access(model_file, os.W_OK):
    os.system('wget http://places2.csail.mit.edu/models_places365/' + model_file)


# from functools import partial
# import pickle
# pickle.load = partial(pickle.load, encoding="latin1")
# pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
# model = torch.load(model_file, map_location=lambda storage, loc: storage, pickle_module=pickle)

model = models.alexnet(num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

In [0]:
import torch.nn as nn
def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf

def myforward(model, x):
    m1 = nn.Sequential(*list(model.children())[:-1])
    m2 = nn.Sequential(*list(model.children())[2][:2])
    feature = m1(x)
    feature = torch.flatten(feature, start_dim=0, end_dim=-1)
    feature = m2(feature)
    feature = feature.view(x.size(0), -1)
    return feature


# forward image
tf = returnTF()

In [0]:
x = "123.png"
savepath = "/content/2014-12-09-13-21-02/"
savepath+x[:-4]+'.mat'

'/content/2014-12-09-13-21-02/123.mat'

In [0]:
import scipy.io as io
import os
filepath = "/content/Downloads/2015-10-29-12-18-17/stereo/left/"
savepath = "/content/2015-10-29-12-18-17/"
 
for filename in os.listdir(filepath):
    img = cv2.imread(filepath+filename,cv2.IMREAD_GRAYSCALE)
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))
    input_img = V(tf(img).unsqueeze(0))
    output = myforward(model, input_img)
    x = output.data.numpy()
    io.savemat(savepath+filename[:-4]+'.mat', {'fc6': x}) 


In [16]:
!zip -r 2015-10-29-12-18-17.zip /content/2015-10-29-12-18-17/

Streaming output truncated to the last 5000 lines.
  adding: content/2015-10-29-12-18-17/1446121277256611.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121394882732.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121282569173.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121246193814.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121327632093.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121353694836.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121218818567.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121269819037.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121335694671.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121392507709.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121180505712.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121383820134.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121137692822.mat (deflated 8%)


In [0]:
from google.colab import files
files.download('2014-12-10-18-10-50.zip')
files.download('2014-12-16-09-14-09.zip')
files.download('2015-02-03-08-45-10.zip')
files.download('2015-05-19-14-06-38.zip')

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!rm -rf /content/Downloads/2014-12-09-13-21-02